## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: H1
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [10]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [12]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [13]:
cell_line = CELL_LINES[2]
cell_line

'H1'

---

In [14]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [15]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [16]:
task = TASKS[0]

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [7]:
model=EmbraceNetMultimodal

In [10]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:57,323] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1583061350627056
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 512
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    lr: 0.02486940405656739
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.9936028132383526
    weight_decay: 0.0003286850904460199

===============> MODEL TESTING
AUPRC test score: 0.0459217762

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:00,264] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.08511619695377697
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 32
    lr: 0.0045407483542726015
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.1202684868716477
    weight_decay: 0.03409622407483309

===============> MODEL TESTING
AUPRC test score: 0.05050148957298904


>>> ITERATION N. 3

============

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:01,373] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.06705766901779102
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 256
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 32
    lr: 0.0036704642162835913
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.9856054197201917

In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [10]:
model=ConcatNetMultimodal

In [14]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:02,856] Using an existing study with name 'H1_active_E_vs_inactive_E_ConcatNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.07154615473725563
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    lr: 1.7173297395160127e-05
    optimizer: Adam
    weight_decay: 0.07224131811143189

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:03,692] Using an existing study with name 'H1_active_E_vs_inactive_E_ConcatNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.06790862547980872


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.07270175550719195
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    lr: 6.652867023288133e-05
    optimizer: Adam
    weight_decay: 0.0011066045473664783

===============> MODEL TESTING
AUPRC test score: 0.07771727911422795




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:05,155] Using an existing study with name 'H1_active_E_vs_inactive_E_ConcatNetMultimodal_3' instead of creating a new one.


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.08733389095083774
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 256
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.5
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 32
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 0.04195477246455094
    optimizer: Adam
    weight_decay: 0.002109

In [15]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [16]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [17]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (28200) in class 0 will be larger than the number of samples in the majority class (class #0 -> 26716)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:54:51,110] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.11033225902466835
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 4
    lr: 0.0008029006712408967
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.3720812450233796
    weight_decay: 0.00016037218011088518

===============> MODEL TESTING
AUPRC test score: 0.14894462501676783


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (26846) in class 0 will be larger than the number of samples in the majority class (class #0 -> 26806)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:55:33,135] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (40289) in class 0 will be larger than the number of samples in the majority class (class 

Study statistics: 
  Number of finished trials:  9
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.13945716613485898
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.00034646869555732653
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.8805943373307533
    weight_decay: 0.0008448976309638311

===============> MODEL TESTING
AUPRC test score: 0.1578628086127522


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (26815) in class 0 will be larger than the number of samples in the majority class (class #0 -> 26796)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:56:17,804] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (40660) in class 0 will be larger than the number of samples in the majority class (class 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1472356251665934
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    lr: 4.188980635244006e-05
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5292121012521165
    weight_decay: 0.000264288510567007

===============> MODEL TESTING
AUPRC test score: 0.15117064496719518



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.15266


In [11]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [20]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [10]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [14]:
model=EmbraceNetMultimodal

In [20]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:07,249] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2372670635438047
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 128
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.00021882365019252332
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.2360790146002768
    weight_decay: 0.0025671170099433567

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:07,925] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.2629483217720976


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.22545575645000787
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.00024302759942883322
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.16337855919376854
    weight_decay: 0.025209674128879034

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:08,721] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_3' instead of creating a new one.


AUPRC test score: 0.13207414957319408


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.30295108695732964
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 128
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.0005959894370344989
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.9899401817896798
    weight_decay: 0.0007799107359551361

===============> MODEL TESTING
AUPRC test score: 0.29792313411857274



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.23098


In [21]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [22]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [17]:
model=ConcatNetMultimodal

In [24]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:10,237] Using an existing study with name 'H1_active_P_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2963443953489828
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.2
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 256
    CONCATNET_n_units_l2: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.003676028579958433
    optimizer: Adam
    weight_decay: 0.0014882689850376795

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:10,910] Using an existing study with name 'H1_active_P_vs_inactive_P_ConcatNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.28916383409996743


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.30668408590153085
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.3
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 64
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0068879610205733995
    optimizer: Nadam
    weight_decay: 0.00021885507037223777

===============> MODEL TESTING
AUPRC test score: 0.3129486274958631


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:11,706] Using an existing study with name 'H1_active_P_vs_inactive_P_ConcatNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2824689044438245
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    CONCATNET_dropout_l0: 0.3
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 768
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.011149644817234438
    optimizer: RMSprop
    weight_decay: 0.016737776445462704

===============> MODEL TESTING
AUPRC test score: 0.28246579264220184



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.29486


In [25]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [26]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76940) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38470)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:57:15,926] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (115532) in class 0 will be larger than the number of samples in the majority class (class

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2945696508118481
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    lr: 0.00335218464492852
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.37547817825011987
    weight_decay: 0.0009472749232261634

===============> MODEL TESTING
AUPRC test score: 0.2955594158304858


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76988) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38494)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:58:33,216] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (115316) in class 0 will be larger than the number of samples in the majority class (class

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.25877114351364466
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 128
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 32
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 4
    lr: 6.124626095553878e-05
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.3438023255434157
    weight_decay: 0.0004132115377483892

===

/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76846) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38423)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 13:59:58,329] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (115360) in class 0 will be larger than the number of samples in the majority class (class

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.29254854254868595
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0021137759951126134
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.6008614144963991
    weight_decay: 0.0006686312536673431

===============> MODEL TESTING
AUPRC test score: 0.3051541013942146



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.24426


In [17]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [20]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [12]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [30]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:12,527] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4940241027617981
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    lr: 0.002759240431928822
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.18166653723157233
    weight_decay: 0.004156884416782315

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:13,260] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.5048360553374328


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4132848663805836
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    FFNN_n_units_l3: 4
    lr: 8.28636365544724e-05
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.9024914636724647
    weight_decay: 0.01549534419174572

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:14,170] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_3' instead of creating a new one.


AUPRC test score: 0.3867365820428389


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21237003173719807
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    lr: 0.0003141795331858006
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.2937643981600543
    weight_decay: 0.0034838148827181685

===============> MODEL TESTING
AU

In [31]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [32]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [24]:
model=ConcatNetMultimodal

In [34]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:15,100] Using an existing study with name 'H1_active_E_vs_active_P_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.40940273737534005
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 64
    lr: 3.3676761510143735e-05
    optimizer: RMSprop
    weight_decay: 0.005312276167792979

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:15,968] Using an existing study with name 'H1_active_E_vs_active_P_ConcatNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.49581219979098606


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5884672414677029
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.3
    CONCATNET_dropout_l2: 0.0
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 32
    CONCATNET_n_units_l2: 64
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 8.778291339632919e-05
    optimizer: RMSprop
    weight_decay: 0.00012207415224986024

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:16,739] Using an existing study with name 'H1_active_E_vs_active_P_ConcatNetMultimodal_3' instead of creating a new one.


AUPRC test score: 0.27134313125113413


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5584197384478445
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.0
    CONCATNET_dropout_l2: 0.2
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 32
    CONCATNET_n_units_l2: 128
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 64
    lr: 0.00024998964233057897
    optimizer: Nadam
    weight_decay: 0.0012122309125247332

===============> MODEL TESTING
AUPRC test score: 0.5097643127817434



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.42564


In [35]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [36]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [49]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11894) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5947)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 14:01:01,618] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (17848) in class 0 will be larger than the number of samples in the majority class (class #0 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3290143351655668
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 32
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 64
    FFNN_n_units_l3: 16
    lr: 0.007033478434306904
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.9601782500515569
    weight_decay: 0.0016138975673270048

===============> MODEL TESTING
AUPRC test score: 0.2510947032374491


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11810) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5905)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 14:01:04,178] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (17718) in class 0 will be larger than the number of samples in the majority class (class #0 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.45384913959080914
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 256
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 32
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    lr: 7.488622180923506e-05
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.7748417987588019
    weight_decay: 0.012546881026093285

===============> MODEL TESTING
AUPRC test score: 0.29751488223884454


>>> ITERATION N. 3

===============> HYPERPARAMETE

/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11834) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5917)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 14:01:06,706] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (17750) in class 0 will be larger than the number of samples in the majority class (class #0 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4573377129900078
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 256
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 64
    lr: 0.0007506221153993081
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.18683749763726376
    weight_decay: 0.020348979070726424

===============> MODEL TESTING
AUPRC test score: 0.3483064625112293



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.29897


In [23]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [52]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [14]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [28]:
model=EmbraceNetMultimodal

In [40]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


[I 2021-11-05 08:44:18,803] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6039113034707507
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 256
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 0.00015918127595460435
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.93766761

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:19,274] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5629252779006668
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 128
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 16
    FFNN_n_units_l3: 4
    lr: 0.009915376440636487
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.5018508545035668
    weight_decay: 0.006850933466532975

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:19,675] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_3' instead of creating a new one.


AUPRC test score: 0.5106541154384525


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5636122765238922
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 32
    lr: 0.00940444924445015
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.01855564444831359
    weight_decay: 0.00023176586544573432

===============> MODEL TESTING
AUPRC test score: 0.5673891667143186



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 

In [41]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [42]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [31]:
model=ConcatNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:24,198] Using an existing study with name 'H1_inactive_E_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5775215394895898
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 64
    lr: 0.00010964844227411522
    optimizer: RMSprop
    weight_decay: 0.040446641153436304

===============> MODEL TESTING
AUPRC test score: 0.5711555042509762




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:24,721] Using an existing study with name 'H1_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5956298509136957
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    lr: 0.004833697425185683
    optimizer: RMSprop
    weight_decay: 0.0002849751081951436

===============> MODEL TESTING
AUPRC test score: 0.584887174297154


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:25,191] Using an existing study with name 'H1_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5956723766860531
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.5
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 64
    CONCATNET_n_units_l2: 128
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 64
    FFNN_n_units_l3: 4
    lr: 0.00012922915138643924
    optimizer: Adam
    weight_decay: 0.0036913675871937477

===============> MODEL TESTING
AUPRC test score: 0.5884931476309033



3-FOLD CROSS-VALIDATION AUPR

In [23]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [53]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (77030) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38515)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (53458) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38515)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:46:04,324] Using an exist

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.573568205613774
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 4
    lr: 0.00011595745198858912
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.03973755596136075
    weight_decay: 0.018144469674786022

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (115538) in class 0 will be larger than the number of samples in the majority class (class #0 -> 57769)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (80194) in class 1 will be larger than the number of samples in the majority class (class #0 -> 57769)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5713873684650889


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

AUPRC test score: 0.5808812438482758


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (77032) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38516)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (53456) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38516)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:50:49,033] Using an exist

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5812121147541082
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 4
    lr: 2.0236919470440772e-05
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.3321223541846935
    weight_decay: 0.002660722895414048

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (115540) in class 0 will be larger than the number of samples in the majority class (class #0 -> 57770)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (80192) in class 1 will be larger than the number of samples in the majority class (class #0 -> 57770)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5899841570653451



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.58075


In [11]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [56]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [16]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [35]:
model=EmbraceNetMultimodal

In [46]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:21,517] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.19885253177572176
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    lr: 0.0014204089282033705
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.17760481212101187
    weight_decay: 0.006056630119860941

===============> MODEL TESTING
AUPRC test score: 0.1878100673525111




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:22,347] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.19254512865615558
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 64
    lr: 0.00011445985493502701
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.10119750877770672
    weight_decay: 0.031225412688850526

===============> MODEL TESTING
AUPRC test score: 0.19901219709439782


>>> ITERATION N. 3

======

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:44:22,949] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.26871990258820644
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 128
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 32
    lr: 0.0008547623303846789
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.7770202175797156
    weight_decay: 0.00018474425276196314

===============> MODEL TESTING
AUPRC test score: 0.26811879203217376



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.21831


In [47]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [48]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [38]:
model=ConcatNetMultimodal

In [28]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:30,037] Using an existing study with name 'H1_active_EP_vs_inactive_rest_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20639336886891996
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 128
    CONCATNET_n_units_l2: 32
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.013071612649570232
    optimizer: RMSprop
    weight_decay: 0.0018595715960715358

===============> MODEL TESTING
AUPRC test score: 0.24894764953395837


>>> ITERATION N. 2


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:30,508] Using an existing study with name 'H1_active_EP_vs_inactive_rest_ConcatNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2249469960074686
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 96
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 1.5501788161810762e-05
    optimizer: RMSprop
    weight_decay: 0.0001003999327354702

===============> MODEL TESTING
AUPRC test score: 0.2731025822155888


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 13:20:30,938] Using an existing study with name 'H1_active_EP_vs_inactive_rest_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20689597826408362
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.3
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    lr: 0.0004235650098953937
    optimizer: Adam
    weight_decay: 0.0002844925253592661

===============> MODEL TESTING
AUPRC test score: 0.16059185073472335



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.22755


In [29]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [30]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [57]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (130578) in class 0 will be larger than the number of samples in the majority class (class #0 -> 65289)
  f"After over-sampling, the number of samples ({n_samples})"


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:53:53,509] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2519723965435893
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 1.8929385325577277e-05
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.8347004174557457
    weight_decay: 0.0010345315708294744

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (195898) in class 0 will be larger than the number of samples in the majority class (class #0 -> 97949)
  f"After over-sampling, the number of samples ({n_samples})"


...

AUPRC test score: 0.2616355966289672


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (130406) in class 0 will be larger than the number of samples in the majority class (class #0 -> 65203)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:02:41,055] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation_2_augmentation_3_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1898187616002604
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 32
    lr: 2.4963774307118384e-05
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5377659233766763
    weight_decay: 0.02712589263012695

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (195584) in class 0 will be larger than the number of samples in the majority class (class #0 -> 97792)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.18988419773889575



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.24059


In [17]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## NO FREEZED LAYERS

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## FREEZED LAYERS + BO + no augmentation

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

In [ ]:
plot_scores(cells=cell_line)